In [1]:
##Robs Static parameter getter
import pandas as pd
import os
import datetime


mainPath = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth'
paramTableFileName = 'ParameterTable.csv'
fuAreaTableFileName = 'fuAreasTable.csv'

outputPath = r'P:\projects\RC10_ResultsSets\FineSedParameters'

#regList = ['BU']
regList = ['BM','BU','CY','FI','MW','WT']

constituentColName = 'CONSTITUENT'
constituentList = ['Sediment - Fine']

catchFuCombo = 'CATCHFU'


usleAndGullyModelFull = 'Sediment Generation (USLE & Gully) - SedNet'
emcAndGullyModelFull = 'Sediment Generation (EMC & Gully) - SedNet'
cropAndGullyModelFull = 'Cropping Sediment (Sheet & Gully) - GBR'
usleAndGullyModelHill = 'Hillslope Generation - USLE'
emcAndGullyModelHill = 'Hillslope Generation - EMC'
cropAndGullyModelHill = 'Hillslope Generatiom - CROP'
gullyGeneration = 'Gully Generation'
inStreamFineSedModel = 'In Stream Fine Sediment Model - SedNet'
inStreamShort = 'In-stream processing'
lewisModel = 'Lewis Trapping Model - GBR'
trappingShort = 'Storage Trapping'


#percentSub = Sub-surface contribution, but they don't both always get written out????

usleHillParams = ['percentSub', 'R Factor Rainfall Threshold', 'Alpha', 'Mean Summer Rainfall', 'Mean Annual Rainfall',
                      'USLE Dry Weather Concentration', 'USLE Maximum Fine Sediment Allowable Runoff Concentration',
                      'USLE HSDR Fine', 'USLE HSDR Coarse', 'USLE Average Fine Particle %']

emcHillParams = ['percentSub', 'Fine Sediment EMC', 'Fine Sediment DWC', 'Coarse Sediment EMC', 'Coarse Sediment DWC']

cropHillParams = ['Sub-surface contribution', 'Hillslope Sediment Delivery Ratio - Fine Sediment', 'Hillslope Sediment Delivery Ratio - Coarse Sediment',
                      'Surface Percentage Clay + Silt', 'Hillslope SlowFlow Concentration - Fine Sediment',
                      'Hillslope SlowFlow Concentration - Coarse Sediment', 'Load Conversion Factor']

gullyParams = ['Gully SDR - Fine', 'Gully SDR - Coarse', 'Total Gully Volume', 'Gully Soil Bulk Density',
               'Gully Clay + Silt Percentage', 'Gully Density', 'Gully Cross Section Area', 'Gully Annual Average Sediment Supply',
              'Average Gully Activity Factor', 'Gully Management Practice Factor', 'Year Of Disturbance', 'Year Of Gully Density Raster',
               'End Year Of Gully']



inStreamFineSedParams = ['Flood Plain Area', 'Bank Full Flow', 'Fine Sediment Settling Velocity For Flood Plains',
                         'Fine Sediment Settling Velocity For Channel Deposition',
                         'Fine Sediment Settling Velocity For Channel Remobilisation', 'Long Term Average Daily Flow ^ Factor',
                         'Daily Flow Power Factor', 'Annual Return Interval', 'Link Slope', 'Link Length', 'Channel Width',
                         'Channel Depth', 'Bank Height', 'Contributing Area', 'Mannings N', 'Riparian Vegetation Percentage',
                         'Maximum Riparian Vegetation Effectiveness', 'Soil Erodibility', 'Soil Percent Fine Sediment',
                         'Bank Erosion Coefficent', 'Sediment Bulk Density',
                         'Bank Height Proportion For Fine Sediment Deposition',
                         'Stream bed channel store initial proportion For Fine Sediment Deposition',
                         'Bank Erosion Management Factor']

lewisParams = ['Length of Reservoir', 'Subtractor Parameter', 'Multiplier Parameter', 'Length/Discharge Factor',
               'Length/Discharge Power']

#modelTypeList = {'Hillslope':{usleAndGullyModelFull:usleHillParams, emcAndGullyModelFull:emcHillParams, cropAndGullyModelFull:cropHillParams},
#                 {'Gully':{gullyGeneration:gullyParams}}, {'Stream':{inStreamShort:inStreamFineSedParams}}, {'Trapping':{trappingShort:lewisParams}}}

modelsAndParams = {}


In [2]:
if not os.path.exists(outputPath):
    os.makedirs(outputPath)
    print("Made dir: " + outputPath)


In [3]:
def getPathInfo(regionIDString):
    paramTablePath = mainPath + '\\' + regionIDString + '\\Model_Outputs\\' + regionIDString + '_BASE_RC10\\' + paramTableFileName
    fuAreasTablePath = mainPath + '\\' + regionIDString + '\\Model_Outputs\\' + regionIDString + '_BASE_RC10\\' + fuAreaTableFileName
    #return fileIn, fileOut
    return paramTablePath, fuAreasTablePath

In [4]:
#Concatenate catch/FU from Parameter table
def combineCatchFUParams(row):
    
    return row['CATCHMENT'] + "|" + row['ELEMENT']



In [5]:
def grabHillslopeParameters(regString, allParams, relevantFUs):
    
    #filter by constituent
    filtered1 = allParams.loc[allParams[constituentColName].isin(constituentList)]
    
    #filter by FUs with non zero area
    filtered2 = filtered1.loc[filtered1[catchFuCombo].isin(relevantFUs)]
    
    #drop LINK col
    #where 1 is the axis number (0 for rows and 1 for columns.)
    filtered2 = filtered2.drop('LINK', 1)
    
    #USLE   
    filteredUSLEall = filtered2.loc[filtered2['MODEL'].isin([usleAndGullyModelFull])]
    filteredUSLEfinal = filteredUSLEall.loc[filteredUSLEall['PARAMETER'].isin(usleHillParams)]
    filteredUSLEfinal = filteredUSLEfinal.drop(catchFuCombo, 1)
    filteredUSLEfinal.to_csv(os.path.join(outputPath, regString + "_FineSed_USLEParams.csv"), index=False)
    
    print("Saved Hillslope USLE parameters for " + regID + " region")
    
    #EMC   
    filteredEMCEall = filtered2.loc[filtered2['MODEL'].isin([emcAndGullyModelFull])]
    filteredEMCfinal = filteredEMCEall.loc[filteredEMCEall['PARAMETER'].isin(emcHillParams)]
    filteredEMCfinal = filteredEMCfinal.drop(catchFuCombo, 1)
    filteredEMCfinal.to_csv(os.path.join(outputPath, regString + "_FineSed_EMCParams.csv"), index=False)
    
    print("Saved Hillslope EMC parameters for " + regID + " region")
    
    #EMC   
    filteredCROPall = filtered2.loc[filtered2['MODEL'].isin([cropAndGullyModelFull])]
    filteredCROPfinal = filteredCROPall.loc[filteredCROPall['PARAMETER'].isin(cropHillParams)]
    filteredCROPfinal = filteredCROPfinal.drop(catchFuCombo, 1)
    filteredCROPfinal.to_csv(os.path.join(outputPath, regString + "_FineSed_CROPParams.csv"), index=False)
    
    print("Saved Hillslope CROP parameters for " + regID + " region")
    
    return

In [6]:
def grabGullyParameters(regString, allParams, relevantFUs):
    
    #filter by constituent
    filtered1 = allParams.loc[allParams[constituentColName].isin(constituentList)]
    
    #filter by FUs with non zero area
    filtered2 = filtered1.loc[filtered1[catchFuCombo].isin(relevantFUs)]
    
    #drop LINK col
    #where 1 is the axis number (0 for rows and 1 for columns.)
    filtered2 = filtered2.drop('LINK', 1)
    
    #GULLY  
    filteredGULLYfinal = filtered2.loc[filtered2['PARAMETER'].isin(gullyParams)]
    filteredGULLYfinal = filteredGULLYfinal.drop(catchFuCombo, 1)
    filteredGULLYfinal.to_csv(os.path.join(outputPath, regString + "_FineSed_GULLYParams.csv"), index=False)
    
    print("Saved Gully parameters for " + regID + " region")
    
    
    return

In [7]:
def grabInstreamParameters(regString, allParams):
    
    #filter by constituent
    filtered1 = allParams.loc[allParams[constituentColName].isin(constituentList)]
    
    filteredStreamAll = filtered1.loc[filtered1['MODEL'].isin([inStreamFineSedModel])]
    filteredStreamFinal = filteredStreamAll.loc[filteredStreamAll['PARAMETER'].isin(inStreamFineSedParams)]
    filteredStreamFinal = filteredStreamFinal.drop(catchFuCombo, 1)
    filteredStreamFinal.to_csv(os.path.join(outputPath, regString + "_FineSed_InStreamParams.csv"), index=False)
    
    print("Saved InStream parameters for " + regID + " region")
    
    return

In [8]:
def grabTrappingParameters(regString, allParams):
    
    #filter by constituent
    filtered1 = allParams.loc[allParams[constituentColName].isin(constituentList)]
    
    filteredTrappingAll = filtered1.loc[filtered1['MODEL'].isin([lewisModel])]
    filteredTrappingFinal = filteredTrappingAll.loc[filteredTrappingAll['PARAMETER'].isin(lewisParams)]
    filteredTrappingFinal = filteredTrappingFinal.drop(catchFuCombo, 1)
    filteredTrappingFinal.to_csv(os.path.join(outputPath, regString + "_FineSed_TrappingParams.csv"), index=False)
    
    print("Saved Trapping parameters for " + regID + " region")
    
    return

In [9]:
#Concatenate catch/FU from Areas table
def combineCatchFUAreas(row):
    
    return row['Catchment'] + "|" + row['FU']



In [10]:
#Grab the parameter table and FU areas first

for regID in regList:
    
    paramTablePath = mainPath + '\\' + regID + '\\Model_Outputs\\' + regID + '_BASE_RC10\\' + paramTableFileName
    fuAreasTablePath = mainPath + '\\' + regID + '\\Model_Outputs\\' + regID + '_BASE_RC10\\' + fuAreaTableFileName
    
    print("Getting parameters and FU areas for " + regID + " region " + str(datetime.datetime.now()))
    
    allParamsDF = pd.read_csv(paramTablePath, error_bad_lines=False)
    fuAreasDF = pd.read_csv(fuAreasTablePath, error_bad_lines=False)
    
    print("Have parameters and FU areas for " + regID + " region")
    
    #Consolidate FU info
    print("Consolidating FU info for " + regID + " region")
    
    
    allParamsDF[catchFuCombo] = allParamsDF.apply(combineCatchFUParams, axis=1)
    
    #print(fuAreasDF.head(20))
    
    filteredFUs = fuAreasDF[fuAreasDF['Area'] > 0]
    
    filteredFUs[catchFuCombo] = filteredFUs.apply(combineCatchFUAreas, axis=1)
    
    fuList = filteredFUs[catchFuCombo].tolist()
    
    #print(filteredFUs)
    #print(fuList)
    
    print("Doing Hillslope for " + regID + " region")
    grabHillslopeParameters(regID, allParamsDF, fuList)
    
    print("Doing Gully for " + regID + " region")
    grabGullyParameters(regID, allParamsDF, fuList)

    print("Doing In-stream for " + regID + " region")
    grabInstreamParameters(regID, allParamsDF)
    
    print("Doing trapping for " + regID + " region")
    grabTrappingParameters(regID, allParamsDF)

print("Finished first summary " + str(datetime.datetime.now()))



Getting parameters and FU areas for BM region 2020-08-28 14:05:08.886600
Have parameters and FU areas for BM region
Consolidating FU info for BM region


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Doing Hillslope for BM region
Saved Hillslope USLE parameters for BM region
Saved Hillslope EMC parameters for BM region
Saved Hillslope CROP parameters for BM region
Doing Gully for BM region
Saved Gully parameters for BM region
Doing In-stream for BM region
Saved InStream parameters for BM region
Doing trapping for BM region
Saved Trapping parameters for BM region
Getting parameters and FU areas for BU region 2020-08-28 14:05:21.423486
Have parameters and FU areas for BU region
Consolidating FU info for BU region
Doing Hillslope for BU region
Saved Hillslope USLE parameters for BU region
Saved Hillslope EMC parameters for BU region
Saved Hillslope CROP parameters for BU region
Doing Gully for BU region
Saved Gully parameters for BU region
Doing In-stream for BU region
Saved InStream parameters for BU region
Doing trapping for BU region
Saved Trapping parameters for BU region
Getting parameters and FU areas for CY region 2020-08-28 14:05:47.574929
Have parameters and FU areas for CY r

In [ ]:
#Hillslope first
